How to use the mixture_of_experts module. 

Thanks for simple example [github.com/yunjey/pytorch-tutorials](https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/01-basics/feedforward_neural_network/main.py)

In [1]:
from mixture_of_experts import mixture_of_experts

In [2]:
# Set up network composer.

class experts(mixture_of_experts):
    
    def compose(self, layers):
        return nn.Sequential(*layers)

In [3]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 3
batch_size = 100
learning_rate = 0.001

# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='./data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

# Mixture of Networks
model = experts(
    input_dim=input_size,
    output_dim=num_classes,
    k=2,
    experts=[
        {
            'n': 2,
            'layers': [
                nn.Linear(input_size, hidden_size),
                nn.Sigmoid(),
                nn.Linear(hidden_size, num_classes),
            ],
        },
        {
            'n': 2,
            'layers': [
                nn.Linear(input_size, hidden_size),
                nn.ReLU(),
                nn.Linear(hidden_size, num_classes),
            ],
        },
        {
            'n': 2,
            'layers': [
                nn.Linear(input_size, hidden_size),
                nn.Tanh(),
                nn.Linear(hidden_size, num_classes),
            ],
        },
    ]
).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs, mloss = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        (loss + mloss).backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


Epoch [1/3], Step [100/600], Loss: 0.2039
Epoch [1/3], Step [200/600], Loss: 0.4369
Epoch [1/3], Step [300/600], Loss: 0.3419
Epoch [1/3], Step [400/600], Loss: 0.1574
Epoch [1/3], Step [500/600], Loss: 0.0756
Epoch [1/3], Step [600/600], Loss: 0.1699
Epoch [2/3], Step [100/600], Loss: 0.2235
Epoch [2/3], Step [200/600], Loss: 0.1652
Epoch [2/3], Step [300/600], Loss: 0.0689
Epoch [2/3], Step [400/600], Loss: 0.2040
Epoch [2/3], Step [500/600], Loss: 0.0671
Epoch [2/3], Step [600/600], Loss: 0.1052
Epoch [3/3], Step [100/600], Loss: 0.0532
Epoch [3/3], Step [200/600], Loss: 0.0391
Epoch [3/3], Step [300/600], Loss: 0.0546
Epoch [3/3], Step [400/600], Loss: 0.1116
Epoch [3/3], Step [500/600], Loss: 0.0837
Epoch [3/3], Step [600/600], Loss: 0.0644


In [4]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs, _ = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 97.38 %
